In [51]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
# %pip install py_vollib
import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
import py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 252 # T = time to expiration in years, 252 trading days
df = df.drop(["date"], axis=1)
df.head()
print(df)

#interest rate data:
df2 = pd.read_csv('/Users/martinlaukkonen/hedging/hedging/code/csv-data/daily-treasury-rates.csv')
three_month_rate = df2['13 WEEKS COUPON EQUIVALENT']
date = df2['Date'] 
print(date, three_month_rate)





          Date  Underlying   C170   C175   C180   C185  C190  P170  P175  \
0   2023-08-21      175.84  16.40  13.27  10.30   7.85  5.91  6.81  8.65   
1   2023-08-22      177.23  17.50  14.10  11.10   8.51  6.34  6.26  8.10   
2   2023-08-23      181.12  20.20  16.65  13.45  10.55  8.08  5.25  6.71   
3   2023-08-24      176.38  16.97  13.75  10.80   8.35  6.25  6.85  8.62   
4   2023-08-25      178.61  18.50  15.15  11.85   9.19  6.90  5.90  7.50   
..         ...         ...    ...    ...    ...    ...   ...   ...   ...   
58  2023-11-10      186.40  19.65  15.35  11.37   7.88  5.00  1.31  1.98   
59  2023-11-13      184.80  18.18  13.86  10.07   6.80  4.16  1.46  2.24   
60  2023-11-14      187.44  20.37  15.84  11.82   8.16  5.27  0.97  1.56   
61  2023-11-15      188.01  20.75  16.25  12.30   8.55  5.50  0.92  1.42   
62  2023-11-16      189.71  22.22  17.80  13.56   9.80  6.40  0.86  1.29   

     P180   P185   P190         T  
0   10.85  13.35  16.50  0.599206  
1   10.10  12.7

The delta measures the rate of change of the security wrt. to the price of the underlying. Thus, the delta of the stock is one.
Vega measures the rate of change of the security wrt. to the volatility of the underlying. Again, the vega of the stock is one. In delta hedging, we short the option and hold a dynamically adjusted amount of the risky asset to eliminate the risk.

Next, calculate the hedge with 2 portfolios: OP and RE. OP has a long call and RE has a short amount of the underlying asset (here, $AAPL). We want P = OP + RE to be neutral to small changes in the underlying price.

# Delta hedging a single option

In [52]:
# The code below is bit of spaghetti code and could use some significant improvements when time
# TODO:
# - give all necessary information (option amounts, strikes, option types, etc.) as a parameter
# - handle printing in its own function
# - save and show the data in a clearer format
def delta_hedge(call_option_prices):
    option_amount = 1000
    interest_rate = 0.05076 # 3-Month Treasury Yield
    option_type = 'call'
    stock_price = call_option_prices.iloc[0]['Underlying']
    option_price = call_option_prices.iloc[0]['C180']
    strike = 180
    time_to_maturity = call_option_prices.iloc[0]['T']
    volatility = implied_volatility(price=option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')
    
    # Initial stuff at t = 0
    bsm_obj = BSM(kind=option_type, S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=volatility)
    OP = bsm_obj.value() # Value of the option according to BSM
    RE = - bsm_obj.delta() * stock_price
    delta = bsm_obj.delta()

    print("Initial values:")
    print("Date:", call_option_prices.iloc[0]['Date'])
    print("Value of single call option / long position according to BSM: ", round(OP, 2))
    print("Value of the delta of the option according to BSM: ", round(delta, 2))
    print("Value of the short position according to BSM: ", round(RE, 2))
    print('\n')

    total_mean_squared_error = []
    
    # next we calculate changes in OP and RE and let A_0 = d OP - d RE
    # Hedge as long as there are dates in the dataframe
    for i in range(1, len(call_option_prices)-1, 1):
        new_bsm_obj = BSM(kind=option_type, S0=call_option_prices['Underlying'][i], K=strike, T=call_option_prices['T'][i], r=interest_rate, sigma=volatility)
        new_OP = new_bsm_obj.value()
        new_RE = - delta * call_option_prices['Underlying'][i]
        d_OP = new_OP - OP
        d_RE = delta * (call_option_prices['Underlying'][i+1] - call_option_prices['Underlying'][i])
        a_0 = d_OP - d_RE
        total_mean_squared_error.append(a_0**2)

        print('==================================================')
        print("Date:", call_option_prices['Date'][i])
        print("Value of single call option / long position according to BSM: ", round(new_OP, 2))
        print("Value of the delta of the option according to BSM: ", round(delta, 2))
        print("Value of the new short position according to BSM: ", round(new_RE, 2))
        print("Difference of the long positions: ", round(OP - new_OP))
        print("Difference of the short positions: ", round(RE - new_RE))
        print("A_0: ", a_0)
        print('==================================================')
        print('\n')

        OP = new_OP
        RE = new_RE

        # Rehedging happens here
        # every second day, t2, t4, t6, so on..
        # Change the modulo for different intervals, 1 = every day, 30 = every month and so on

        if i % 30 == 0:
            delta = new_bsm_obj.delta()
            # volatility = implied_volatility(price=OP, S=call_option_prices['Underlying'][i], K=strike, t=call_option_prices['T'][i], r=interest_rate, flag='c')
            volatility = BSM(kind=option_type, S0=call_option_prices['Underlying'][i], K=strike, T=call_option_prices['T'][i], r=interest_rate, sigma=volatility).implied_vol(call_option_prices['C180'][i])
            print("Rehedging date:", call_option_prices['Date'][i])
            print("Delta updated, new delta is:", round(delta, 2))
            print("Volatility updated, new implied volatility is:", round(volatility, 5))
            print('To delta-hedge a single long call option, we need to readjust the RE:')
            RE = -delta*call_option_prices['Underlying'][i]
            print('Readjusting replicating portoflio RE, delta * underlying: ', round(RE))
            print('Long position is: ', round(OP)) 
            print('\n')
        
    # Count the total error
    total_error = 0
    for i in range(0, len(total_mean_squared_error)-1):
        total_error += total_mean_squared_error[i]
    total_error = total_error*(1/(len(total_mean_squared_error)-1))
    print("The total mean squared error is : ", total_error)
    

In [53]:
delta_hedge(df)

Initial values:
Date: 2023-08-21
Value of single call option / long position according to BSM:  10.3
Value of the delta of the option according to BSM:  0.55
Value of the short position according to BSM:  -96.32


Date: 2023-08-22
Value of single call option / long position according to BSM:  11.03
Value of the delta of the option according to BSM:  0.55
Value of the new short position according to BSM:  -97.08
Difference of the long positions:  -1
Difference of the short positions:  1
A_0:  -1.4038791517579736


Date: 2023-08-23
Value of single call option / long position according to BSM:  13.31
Value of the delta of the option according to BSM:  0.55
Value of the new short position according to BSM:  -99.21
Difference of the long positions:  -2
Difference of the short positions:  2
A_0:  4.87997028703967


Date: 2023-08-24
Value of single call option / long position according to BSM:  10.45
Value of the delta of the option according to BSM:  0.55
Value of the new short position acco

# Delta-vega hedging a single option

In [54]:
# Copy-paste spaghetti, refactor
def delta_vega_hedge(call_option_prices):
    interest_rate = 0.05076 # 3-Month Treasury Yield
    option_type = 'call'
    stock_price = call_option_prices.iloc[0]['Underlying']
    option_price = call_option_prices.iloc[0]['C170']
    strike = 170
    time_to_maturity = call_option_prices.iloc[0]['T']
    volatility = implied_volatility(price=option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')

    # Initial stuff when t = 0
    bsm_obj = BSM(kind=option_type, S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=volatility)
    OP = bsm_obj.value() # Value of the option according to BSM

    # This needs to be different ??
    # RE = how is this with delta-vega ??
    delta = bsm_obj.delta()
    vega = bsm_obj.vega()
    
    print("Initial values:")
    print("Date:", call_option_prices.iloc[0]['Date'])
    print("Value of single call option / long position according to BSM: ", round(OP, 2))
    print("Value of the Delta of the option according to BSM: ", round(delta, 2))
    print("Value of the Vega of the option according to BSM: ", round(vega, 2))
    print("Value of the implied volatility of option: ", round(volatility, 2))
    print('\n')

    for i in range(1, len(call_option_prices)-1, 1):
        new_bsm_obj = BSM(kind=option_type, S0=call_option_prices['Underlying'][i], K=strike, T=call_option_prices['T'][i], r=interest_rate, sigma=volatility)
        new_OP = new_bsm_obj.value()
        
        
        # This needs to be different ??
        # new_RE = how is this with delta-vega ??
        
        d_OP = new_OP - OP

        # This also needs changing
        # d_RE = delta*(call_option_prices['Underlying'][i+1] - call_option_prices['Underlying'][i])
        # Affects also this
        # a_0 = d_OP - d_RE
        # total_mean_squared_error.append(a_0**2)
    
        print('==================================================')
        print("Date:", call_option_prices['Date'][i])
        print("Value of single call option / long position according to BSM: ", round(new_OP, 2))
        print("Value of the Delta of the option according to BSM: ", round(delta, 2))
        print("Value of the Vega of the option according to BSM: ", round(vega, 2))
        print("Value of the new long position according to BSM: ", round(new_OP))

        print("Difference of the long positions: ", round(d_OP))

        print('==================================================')
        print('\n')
    
        OP = new_OP
        
        #RE = new_RE
            
        # Change modulo to get different hedging frequencies
        if i % 2 == 0:
            delta = new_bsm_obj.delta()
            vega = new_bsm_obj.vega()
            volatility = implied_volatility(price=OP, S=call_option_prices['Underlying'][i], K=strike, t=call_option_prices['T'][i], r=interest_rate, flag='c')
            print("Rehedging date:", call_option_prices['Date'][i])
            print("Delta updated, new Delta is:", round(delta, 2))
            print("Vega updated, new Vega is:", round(vega, 2))
            print("Volatility updated, new implied volatility is:", round(volatility, 2))

            print('\n')



In [55]:
delta_vega_hedge(df)

Initial values:
Date: 2023-08-21
Value of single call option / long position according to BSM:  16.4
Value of the Delta of the option according to BSM:  0.69
Value of the Vega of the option according to BSM:  47.69
Value of the implied volatility of option:  0.19


Date: 2023-08-22
Value of single call option / long position according to BSM:  17.33
Value of the Delta of the option according to BSM:  0.69
Value of the Vega of the option according to BSM:  47.69
Value of the new long position according to BSM:  17
Difference of the long positions:  1


Date: 2023-08-23
Value of single call option / long position according to BSM:  20.15
Value of the Delta of the option according to BSM:  0.69
Value of the Vega of the option according to BSM:  47.69
Value of the new long position according to BSM:  20
Difference of the long positions:  3


Rehedging date: 2023-08-23
Delta updated, new Delta is: 0.76
Vega updated, new Vega is: 43.14
Volatility updated, new implied volatility is: 0.19


Da

In [56]:
# TODO: Spaghetti, refactor
# Straddle = a combination option trading strategy with 1 call option and 1 put option,
# with the same strike price and maturity of the same underlying stock
def delta_hedge_straddle(option_prices):
    option_amount = 1000
    interest_rate = 0.05076 # 3-Month Treasury Yield
    stock_price = option_prices.iloc[0]['Underlying']
    call_option_price = option_prices.iloc[0]['C180']
    put_option_price = option_prices.iloc[0]['P180']
    strike = 180
    time_to_maturity = option_prices.iloc[0]['T']
    call_volatility = implied_volatility(price=call_option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')
    put_volatility = implied_volatility(price=put_option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag='c')
    
    # Initial stuff at t = 0
    call_bsm = BSM(kind='call', S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=call_volatility)
    put_bsm = BSM(kind='put', S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=put_volatility)

    # Forming the straddle portfolio
    put_delta = put_bsm.delta()
    call_delta = call_bsm.delta()
    OP = call_bsm.value() + put_bsm.value() # Value of the option according to BSM
    RE = - call_delta * stock_price + put_delta * stock_price
    combined_delta = call_delta + put_delta

    print("Initial values:")
    print("Date:", option_prices.iloc[0]['Date'])
    print("Value of long Straddle portfolio (call value + put value): ", round(OP, 2))
    print("Value of the delta of the Straddle (call delta + put delta): ", round(delta_combined, 2))
    print("Value of the short Straddle portfolio (-call delta * stock + put delta * stock): ", round(RE, 2))
    print('\n')

    # Keep track of the mean squared error values
    total_mean_squared_error = []
    
    # next we calculate changes in OP and RE and let A_0 = d OP - d RE
    # Hedge as long as there are dates in the dataframe
    for i in range(1, len(option_prices)-1, 1):
        new_call_bsm_obj = BSM(kind='call', S0=option_prices['Underlying'][i], K=strike, T=option_prices['T'][i], r=interest_rate, sigma=call_volatility)
        new_put_bsm_obj = BSM(kind='put', S0=option_prices['Underlying'][i], K=strike, T=option_prices['T'][i], r=interest_rate, sigma=put_volatility)
        new_OP = new_call_bsm_obj.value() + new_put_bsm_obj.value()
        new_RE = -combined_delta * option_prices['Underlying'][i]
        d_OP = new_OP - OP
        d_RE = combined_delta * (option_prices['Underlying'][i+1] - option_prices['Underlying'][i])
        a_0 = d_OP - d_RE
        total_mean_squared_error.append(a_0**2)

        print('==================================================')
        print("Date:", option_prices['Date'][i])
        print("Value of Straddle position: ", round(new_OP, 2))
        print("Value of the delta of the Straddle position: ", round(combined_delta, 2))
        print("Value of the new short position according to BSM: ", round(new_RE, 2))
        print("Difference of the long positions: ", round(OP - new_OP))
        print("Difference of the short positions: ", round(RE - new_RE))
        print("A_0: ", a_0)
        print('==================================================')
        print('\n')

        OP = new_OP
        RE = new_RE

        # Rehedging happens here
        # every second day, t2, t4, t6, so on..
        # Change the modulo for different intervals, 1 = every day, 30 = every month and so on
        if i % 2 == 0:
            combined_delta = new_call_bsm_obj.delta() + new_put_bsm_obj.delta()
            call_volatility = BSM(kind='call', S0=option_prices['Underlying'][i], K=strike, T=option_prices['T'][i], r=interest_rate, sigma=call_volatility).implied_vol(option_prices['C180'][i])
            put_volatility = BSM(kind='put', S0=option_prices['Underlying'][i], K=strike, T=option_prices['T'][i], r=interest_rate, sigma=put_volatility).implied_vol(option_prices['P180'][i])
            print("Rehedging date:", option_prices['Date'][i])
            print("Delta updated, new delta is:", round(combined_delta, 3))
            print("Volatilities updated, new implied volatility for call option is:", round(call_volatility, 5))
            print("Volatilities updated, new implied volatility for put option is:", round(put_volatility, 5))
            print('To delta-hedge a single long call option, we need to readjust the RE:')
            RE = -combined_delta * option_prices['Underlying'][i]
            print('Readjusting replicating portoflio RE, combined delta * underlying: ', round(RE))
            print('Long position is: ', round(OP)) 
            print('\n')
        
    # Count the total error
    total_error = 0
    for i in range(0, len(total_mean_squared_error)-1):
        total_error += total_mean_squared_error[i]
    total_error = total_error*(1/(len(total_mean_squared_error)-1))
    print("The total mean squared error is : ", total_error)
    

In [57]:
delta_hedge_straddle(df)

Initial values:
Date: 2023-08-21
Value of long Straddle portfolio (call value + put value):  19.92


NameError: name 'delta_combined' is not defined